Pull in the packages we need

In [1]:
require(stats)

# Parameters

* **cpd** - Checkins per day. Defaults to 20.

* **days** - Number of days to simulate.  Defaults to 10000, or roughly
    40 working years.

* **winSz** - Size of the commit window in percentage of the working day.
    Defaults to 0.01% with the default for workSec, which is roughly 3
    seconds with the default `workSec` value, a long-ish commit time for Fossil.

* **workSec** - Seconds in working day, defaulting to 8 hours.  This value
    only affects the reporting output, not the results of the underlying
    simulation.  It's a scaling parameter to humanize the results.
    
* **sd** - The standard deviation to use for our normally-distributed random
    numbers. The default gives "nice" distributions, spread over the working
    day with a low chance of generating values outside the working day.
    
    The larger this value gets, the more spread out the checkins, and so the
    lower the chance of collisions.  You might want to increase it a bit to
    simulate early and late workers.  (e.g. `workSec / 3`)
    
    As you decrease this value — e.g. `workSec / 5` — you're simulating a work
    environment where people tend to check their work in closer and closer
    to mid-day, which increases the chance of collision.


In [8]:
collisions <- function(
        cpd = 20,
        days = 10000,
        winSz = 0.01 / 100,
        workSec = 8 * 60 * 60,
        sd = workSec / 4)
{
    cat("Running simulation...\n")

    day = 0
    collisions = 0
    winSec = workSec * winSz
    mean = workSec / 2

    while (day < days) {
        # Create the commit time vector as random values in a normal
        # distribution.
        times = sort(rnorm(cpd, mean, sd))

        # Are there any pairs in the time vector that are within the
        # passed window size?  If so, that's a Fossil checkin collision.
        i = 1
        while (i < cpd) {
            if (abs(times[i] - times[i + 1]) < winSec) {
                collisions = collisions + 1
            }
            i = i + 1
        }
        
        day = day + 1
    }
    
    cat("Found", collisions, "collisions in", days, (workSec / 3600),
        "hour working days with", winSec, "second windows.\n")
}

In [10]:
collisions()

Running simulation...
Found 423 collisions in 10000 8 hour working days with 2.88 second windows.
